In [27]:
import json
import importlib
import pandas as pd
from snorkel.labeling import labeling_function, PandasLFApplier, LFAnalysis
from snorkel.labeling.model import LabelModel

from Project.utils.storage import youtube_db as db

importlib.reload(db)

<module 'Project.utils.storage.youtube_db' from 'C:\\Users\\corti\\Github\\SocialMediaMining\\Project\\utils\\storage\\youtube_db.py'>

In [ ]:
"""
THE YOUTUBE MACHINE LEARNING PIPELINE:

1. Build a DataFrame with [content, podcast_guest, bias, leaning, is_political, sentiment, emotion, llm_label].

2. Extract a sub-DataFrame with only rows that have llm_label.

3. Define Snorkel Labeling Functions

4. Run Snorkel on sub-DataFrame to create 50k labeled instances. We can check which labels were more important.

5. Manually label 150 instances in the entire DataFrame.

6. Create a DataFrame having [content, podcast_guest, bias, leaning, sentiment, emotion, SNORKEL_LABEL].

7. Build a Model using sklearn (Cross Validation and other stuff), training it on the DataFrame on point 6. Check Performance

8. Run the Model on the entire Dataset. Check Performance based on my 150 instances.

At the end, every comment has a label produced by my Classification Model

"""


In [3]:
trump_podcasts = [
    "xrFdHO7FH8w",
    "blqIZGXWUpU",
    "s11uIW7wi-E",
    "vC5cHjcgt5g",
    "G80iLTctFuY",
    "qCbfTN-caFI",
    "Ry1IjOft95c",
    "S7BTdUaNQM8",
    "1SsmPh8gCxU",
    "-dmwG54QsKc",
    "nwQil7tcImI",
    "G9lXnwuZ2qs",
    "hBMoPUAeLnY"
]

harris_podcasts = [
    "_KCRsjPCiCI",
    "bzThwqnQJDY",
    "7L4sts7I3xI",
    "pNbwMrBMGgE",
    "Vu5yD3fu6A8",
]

youtube_db = db.SQLiteYoutubeSaver(db_name='../db/youtube.db')

In [4]:
youtube_db.cursor.execute(f"""
SELECT content, video_id, bias, leaning, is_political, sentiment, emotion, llm_label
FROM CommentAnalysis JOIN Comments on Comments.id = CommentAnalysis.id
""")

data = youtube_db.cursor.fetchall()

df = pd.DataFrame(data)
df.columns = ['content', 'video_id', 'bias', 'leaning', 'is_political', 'sentiment', 'emotion', 'llm_label']
df['podcast_guest'] = ['Trump' if video_id in trump_podcasts else 'Harris' for video_id in df['video_id']]
df.drop('video_id', axis=1, inplace=True)

In [5]:
df

,content,bias,leaning,is_political,sentiment,emotion,llm_label,podcast_guest
0,Best Podcast of All-time? 🤔🔥,RIGHT,RIGHT,0,Positive,joy,Republican,Trump
1,Definitely my favorite!,RIGHT,LEFT,0,Positive,joy,Republican,Trump
2,"No, best guest",RIGHT,RIGHT,0,Positive,joy,Republican,Trump
3,Probably,RIGHT,LEFT,0,Neutral,anticipation,Republican,Trump
4,Under 30 seconds,RIGHT,RIGHT,0,Neutral,anticipation,Neutral,Trump
...,...,...,...,...,...,...,...,...
114695,How do you check the numbers from your phone ...,RIGHT,RIGHT,0,Negative,disgust,None,Harris
114696,@@BruceDragon-sf1tr just google it fam. I prom...,LEFT,RIGHT,0,Neutral,optimism,None,Harris
114697,its now 34k to 88k now lolol,LEFT,LEFT,0,Positive,joy,None,Harris
114698,@@BruceDragon-sf1trjust add it to your browser...,LEFT,RIGHT,0,Neutral,anger,None,Harris


In [16]:
df_with_llm_labels = df[df["llm_label"].notna()].copy()

In [19]:
df_with_llm_labels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52451 entries, 0 to 93638
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   content        52451 non-null  object
 1   bias           52451 non-null  object
 2   leaning        52451 non-null  object
 3   is_political   52451 non-null  int64 
 4   sentiment      52451 non-null  object
 5   emotion        52451 non-null  object
 6   llm_label      52451 non-null  object
 7   podcast_guest  52451 non-null  object
dtypes: int64(1), object(7)
memory usage: 3.6+ MB


In [18]:
df_with_llm_labels

,content,bias,leaning,is_political,sentiment,emotion,llm_label,podcast_guest
0,Best Podcast of All-time? 🤔🔥,RIGHT,RIGHT,0,Positive,joy,Republican,Trump
1,Definitely my favorite!,RIGHT,LEFT,0,Positive,joy,Republican,Trump
2,"No, best guest",RIGHT,RIGHT,0,Positive,joy,Republican,Trump
3,Probably,RIGHT,LEFT,0,Neutral,anticipation,Republican,Trump
4,Under 30 seconds,RIGHT,RIGHT,0,Neutral,anticipation,Neutral,Trump
...,...,...,...,...,...,...,...,...
93634,Curious why she didn’t get called out for not ...,LEFT,RIGHT,1,Negative,anger,Democratic,Harris
93635,"She already thinks she has won. ""When I was Vi...",RIGHT,RIGHT,1,Negative,anticipation,Republican,Harris
93636,Technically she can be the president if she us...,RIGHT,RIGHT,1,Neutral,anticipation,Republican,Harris
93637,That's what people need to realize. If she was...,LEFT,RIGHT,1,Negative,anger,Democratic,Harris


In [9]:
# Label Space
REPUBLICAN = 0
DEMOCRATIC = 1
NEUTRAL = 2
ABSTAIN = -1

In [21]:
@labeling_function()
def lf_llm(x):
    match x.llm_label:
        case "Republican":
            return REPUBLICAN
        case "Democratic":
            return DEMOCRATIC
        case "Neutral":
            return NEUTRAL
        case _:
            return ABSTAIN


@labeling_function()
def lf_bias_left(x):
    match x.bias:
        case "LEFT":
            return DEMOCRATIC
        case "RIGHT":
            return REPUBLICAN if x.podcast_guest == "Trump" else ABSTAIN
    return ABSTAIN


@labeling_function()
def lf_leaning(x):
    if x.is_political == 0: return NEUTRAL
    match x.leaning:
        case "RIGHT":
            return REPUBLICAN
        case "LEFT":
            return DEMOCRATIC
    return ABSTAIN


@labeling_function()
def lf_not_political(x):
    return NEUTRAL if x.is_political == 0 else ABSTAIN


@labeling_function()
def lf_negative_sentiment(x):
    match x.sentiment:
        case "Negative":
            return DEMOCRATIC if x.podcast_guest == "Trump" else REPUBLICAN
        case "Positive":
            return REPUBLICAN if x.podcast_guest == "Trump" else DEMOCRATIC
    return ABSTAIN


@labeling_function()
def lf_emotion(x):
    match x.emotion:
        case "joy":
            return REPUBLICAN if x.podcast_guest == "Trump" else DEMOCRATIC
        case "anger":
            return DEMOCRATIC if x.podcast_guest == "Trump" else REPUBLICAN
    return ABSTAIN


@labeling_function()
def lf_bias_override(x):
    if (x.bias == "RIGHT" and
            x.sentiment == "Positive" and
            x.podcast_guest == "Harris"):
        return DEMOCRATIC
    return ABSTAIN


In [22]:
lfs = [
    lf_llm,
    lf_bias_left,
    lf_leaning,
    lf_not_political,
    lf_negative_sentiment,
    lf_emotion,
    lf_bias_override,
]

applier = PandasLFApplier(lfs)
L = applier.apply(df_with_llm_labels)

100%|██████████| 52451/52451 [00:02<00:00, 21941.02it/s]


In [24]:
LFAnalysis(L, lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
lf_llm,0,"[0, 1, 2]",1.000000,0.999771,0.881203
lf_bias_left,1,"[0, 1]",0.647519,0.647519,0.610837
lf_leaning,2,"[0, 1, 2]",0.984576,0.984576,0.869402
lf_not_political,3,[2],0.440220,0.440220,0.429753
lf_negative_sentiment,4,"[0, 1]",0.907800,0.907800,0.806886
lf_emotion,5,"[0, 1]",0.606013,0.606013,0.539608
lf_bias_override,6,[1],0.051057,0.051057,0.048674


In [28]:
label_model = LabelModel(cardinality=3, verbose=True)
label_model.fit(L_train=L, n_epochs=500, log_freq=100)

INFO:root:Computing O...
INFO:root:Estimating \mu...
100%|██████████| 500/500 [00:00<00:00, 983.00epoch/s] 
INFO:root:Finished Training


In [29]:
snorkel_labels = label_model.predict(L)

In [31]:
df_with_llm_labels["snorkel_labels"] = snorkel_labels

In [32]:
df_with_llm_labels

,content,bias,leaning,is_political,sentiment,emotion,llm_label,podcast_guest,snorkel_labels
0,Best Podcast of All-time? 🤔🔥,RIGHT,RIGHT,0,Positive,joy,Republican,Trump,2
1,Definitely my favorite!,RIGHT,LEFT,0,Positive,joy,Republican,Trump,2
2,"No, best guest",RIGHT,RIGHT,0,Positive,joy,Republican,Trump,2
3,Probably,RIGHT,LEFT,0,Neutral,anticipation,Republican,Trump,2
4,Under 30 seconds,RIGHT,RIGHT,0,Neutral,anticipation,Neutral,Trump,2
...,...,...,...,...,...,...,...,...,...
93634,Curious why she didn’t get called out for not ...,LEFT,RIGHT,1,Negative,anger,Democratic,Harris,0
93635,"She already thinks she has won. ""When I was Vi...",RIGHT,RIGHT,1,Negative,anticipation,Republican,Harris,0
93636,Technically she can be the president if she us...,RIGHT,RIGHT,1,Neutral,anticipation,Republican,Harris,0
93637,That's what people need to realize. If she was...,LEFT,RIGHT,1,Negative,anger,Democratic,Harris,0
